Map Reduce

In [1]:
import pandas as pd
import csv
from functools import reduce
from multiprocessing import Pool

In [2]:
!wget -O housingSalseSample.csv https://github.com/CUSP2021ADS/Data/raw/main/housingSalseSample.csv

--2022-06-04 09:05:33--  https://github.com/CUSP2021ADS/Data/raw/main/housingSalseSample.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CUSP2021ADS/Data/main/housingSalseSample.csv [following]
--2022-06-04 09:05:33--  https://raw.githubusercontent.com/CUSP2021ADS/Data/main/housingSalseSample.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6167298 (5.9M) [text/plain]
Saving to: ‘housingSalseSample.csv’

housingSalseSample. 100%[===================>]   5.88M  --.-KB/s    in 0.02s   

2022-06-04 09:05:33 (255 MB/s) - ‘housingSalseSample.csv’ saved [6167298/6167298]



In [4]:
house = pd.read_csv('housingSalseSample.csv')
house.head()

,BOROUGH,BLOCK,LOT,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,1170,1010,17 CONDOPS,"200 WEST 79TH STREET, 7P",10024.0,0.0,1975.0,551000,2011-07-28
1,4,1275,45,10 COOPS - ELEVATOR APARTMENTS,"35-35 75TH STREET, 304",11372.0,0.0,1941.0,338000,2011-05-20
2,3,5746,58,03 THREE FAMILY DWELLINGS,1249 65TH STREET,11219.0,3861.0,1910.0,1250000,2014-12-04
3,4,2086,50,10 COOPS - ELEVATOR APARTMENTS,"97-07 63 ROAD, 8F",11374.0,0.0,1955.0,253000,2016-03-09
4,4,5028,7,10 COOPS - ELEVATOR APARTMENTS,"38-15 149TH STREET, 1F",11354.0,0.0,1955.0,0,2016-12-30


In [3]:
from functools import reduce
import csv

def returnInfo(record):
    return(record['ZIP CODE'],record['GROSS SQUARE FEET'],record['SALE PRICE'])

def sum_sqft_price(result,record):
    zipcode, sqft, price = record[0], record[1], record[2]
    if zipcode not in result:
        result[zipcode] = (float(sqft),float(price))
    else:
        result[zipcode] = (result[zipcode][0]+float(sqft),result[zipcode][1]+float(price))
    return result

In [5]:
with open('housingSalseSample.csv', 'r') as fi:
    reader = csv.DictReader(fi)

    output = sorted(map(lambda x: [int(float(x[0])),round(x[1],2)],
                 map(lambda x: (x[0],x[1][1]/x[1][0]),
                 list(reduce(sum_sqft_price,
                             filter(lambda x: float(x[1])>500 and float(x[2])>100000,
                                    filter(lambda x: len(x[0])*len(x[1])*len(x[2])>0, 
                                           list(map(returnInfo,reader)))),{}).items()))), key = lambda x: x[1])

In [6]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['zipcode','price']).to_csv('output1.csv',index=False)

grading(output)

In [8]:
from google.colab import files
files.download('output1.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Multiprocessing and Chunk

In [9]:
!wget -O ADSSession6Task2Data.csv https://raw.githubusercontent.com/CUSP2021ADS/Data/main/ADSSession6Task2Data.csv

--2022-06-04 09:11:31--  https://raw.githubusercontent.com/CUSP2021ADS/Data/main/ADSSession6Task2Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279369 (273K) [text/plain]
Saving to: ‘ADSSession6Task2Data.csv’

ADSSession6Task2Dat 100%[===================>] 272.82K  --.-KB/s    in 0.005s  

2022-06-04 09:11:31 (56.1 MB/s) - ‘ADSSession6Task2Data.csv’ saved [279369/279369]



In [10]:
from multiprocessing import Pool

def returnValue(record):
    return (record['Type'],record['Value'],record['Select'])

    
def sumValue(result,record): #add rides from a new record to the dictionary accumilating type-origin-destination-time ridership
    if str(record[2]) == '1':
        result[record[0]] = result.get(record[0],0) + float(record[1])

    return result

In [11]:
with open('ADSSession6Task2Data.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    chunk = []
    result = []
    # read 100000 rows in each chunk
    chunksize = 1000
    pool = Pool(8)
    for i, line in enumerate(reader):
        if (i % chunksize == 0 and i > 0):
            chunkOutput = reduce(sumValue,
                        pool.map(returnValue,chunk),
                        {}).items()
            result += chunkOutput
            chunk = []
            
        chunk.append(line)
        
    chunkOutput = reduce(sumValue,
            pool.map(returnValue,chunk),
            {}).items()
    result += chunkOutput
    pool.close()

In [12]:
def lastReducer(finalRecord,record):
    finalRecord[record[0]] = finalRecord.get(record[0],0) + float(record[1])
    return finalRecord

output = list(reduce(lastReducer,result,{}).items())
output

[('3', 294.32549925839936),
 ('0', 306.7099300824752),
 ('1', 316.7932728146583),
 ('9', 291.68146927692595),
 ('6', 302.75696574193273),
 ('7', 280.6029945361827),
 ('8', 274.5970828385255),
 ('5', 302.4264220421208),
 ('4', 320.753814015207),
 ('2', 286.0854868555854)]

In [13]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output2.csv',index=False)

grading(output)

In [14]:
from google.colab import files
files.download('output2.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>